In [120]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm
import time

In [10]:
wendy_data_directory = '/users/Wendy/Capstone/saudi_arabia_112019_tweets_csv_hashed_1.csv'
tweets = pd.read_csv(wendy_data_directory)

In [12]:
tweets.columns

Index(['tweetid', 'userid', 'user_display_name', 'user_screen_name',
       'user_reported_location', 'user_profile_description',
       'user_profile_url', 'follower_count', 'following_count',
       'account_creation_date', 'account_language', 'tweet_language',
       'tweet_text', 'tweet_time', 'tweet_client_name', 'in_reply_to_userid',
       'in_reply_to_tweetid', 'quoted_tweet_tweetid', 'is_retweet',
       'retweet_userid', 'retweet_tweetid', 'latitude', 'longitude',
       'quote_count', 'reply_count', 'like_count', 'retweet_count', 'hashtags',
       'urls', 'user_mentions'],
      dtype='object')

In [122]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
tweets.head(10)

,tweetid,userid,user_display_name,user_screen_name,user_reported_location,user_profile_description,user_profile_url,follower_count,following_count,account_creation_date,account_language,tweet_language,tweet_text,tweet_time,tweet_client_name,in_reply_to_userid,in_reply_to_tweetid,quoted_tweet_tweetid,is_retweet,retweet_userid,retweet_tweetid,latitude,longitude,quote_count,reply_count,like_count,retweet_count,hashtags,urls,user_mentions,cleaned_text
0,1137497501194493952,PNn2yIomi1HIYwGdduEWQb+Jj4lX5NIHaMQmHrK6Y=,PNn2yIomi1HIYwGdduEWQb+Jj4lX5NIHaMQmHrK6Y=,PNn2yIomi1HIYwGdduEWQb+Jj4lX5NIHaMQmHrK6Y=,المملكة العربية السعودية,أقبل المقدم 💫الريتويت مراقب👀,NaN,3974,5092,2019-02-14,ar,ar,RT @3v5v3: ٢_ 🔴 عسل سدر 🍯 🍯\n\n ❌ ❌ فوائد عسل السدر ❌ ❌\n\n [ للالتهابات ✔\n\n [ الرئوية ✔\n \n[ وأمراض ✔\n\n [ الجهاز التنفسي…,2019-06-08 23:11,Twitter for iPhone,NaN,NaN,NaN,True,NaN,1.137208e+18,absent,absent,0.0,0.0,0.0,0.0,[],[],['915662559264542720'],RT : ٢_ 🔴 عسل سدر 🍯 🍯\n\n ❌ ❌ فوائد عسل السدر ❌ ❌\n\n [ للالتهابات ✔\n\n [ الرئوية ✔\n \n[ وأمراض ✔\n\n [ الجهاز التنفسي…
1,1170605961829990400,1121701880,ấłřấŝŝễễħ_48🕊ḁѦ,suuny2010,Najran🕊 Riyadh🕊🇸🇦lbri🇴🇲,‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏يٌّاٍّرًّبّْ اٍّاٍّرًّحّْمّـُ اٍّمّـُيٌّ وّاٍّاٍّغٌّفّْرًّلَّهّْاٍّاٍّ💔انَثَئ مَعَلَقَهَ بَجَدَاائلَ االَسَمَاء🕊,NaN,9078,2127,2013-01-26,ar,ar,(للرجال نصيب مما ترك الوالدان والأقربون وللنساء نصيب مما ترك الوالدان والأقربون مما قل منه أو كثر نصيبا مفروضا) [النساء:7] https://t.co/QuOqJLHVkO,2019-09-08 07:53,quran twit,NaN,NaN,NaN,False,NaN,NaN,absent,absent,0.0,0.0,1.0,0.0,[],['https://qurantwit.com/tw'],[],(للرجال نصيب مما ترك الوالدان والأقربون وللنساء نصيب مما ترك الوالدان والأقربون مما قل منه أو كثر نصيبا مفروضا) [النساء:7]
2,1172783625445597184,1121701880,ấłřấŝŝễễħ_48🕊ḁѦ,suuny2010,Najran🕊 Riyadh🕊🇸🇦lbri🇴🇲,‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏يٌّاٍّرًّبّْ اٍّاٍّرًّحّْمّـُ اٍّمّـُيٌّ وّاٍّاٍّغٌّفّْرًّلَّهّْاٍّاٍّ💔انَثَئ مَعَلَقَهَ بَجَدَاائلَ االَسَمَاء🕊,NaN,9078,2127,2013-01-26,ar,ar,(من كان يريد حرث الآخرة نزد له في حرثه ومن كان يريد حرث الدنيا نؤته منها وما له في الآخرة من نصيب) [الشورى:20] https://https://t.co/QuOqJLHVkO,2019-09-14 08:06,quran twit,NaN,NaN,NaN,False,NaN,NaN,absent,absent,0.0,0.0,2.0,1.0,[],['https://qurantwit.com/tw'],[],(من كان يريد حرث الآخرة نزد له في حرثه ومن كان يريد حرث الدنيا نؤته منها وما له في الآخرة من نصيب) [الشورى:20]
3,1122861301556969478,245029386,🦋 الًٓــٌٓريــٓـم 🦋,reem78_alfla,Kuwait,Queen of April🦋 instagram : reem_alfla4 snapchat : reem78L تويتر الاحتياطي : reem78alfla,NaN,13141,187,2011-01-30,en,ar,دائما يمثلني @kingoflaw69 👏🏻👏🏻 #البدون #اسقاط_القروض_لليوم_145 https://t.co/BOVN83MXi0,2019-04-29 13:52,Twitter for iPhone,NaN,NaN,NaN,False,NaN,NaN,absent,absent,0.0,9.0,73.0,74.0,"['البدون', 'اسقاط_القروض_لليوم_145']",[],['382706391'],دائما يمثلني 👏🏻👏🏻 #البدون #اسقاط_القروض_لليوم_145
4,1107401207025410049,245029386,🦋 الًٓــٌٓريــٓـم 🦋,reem78_alfla,Kuwait,Queen of April🦋 instagram : reem_alfla4 snapchat : reem78L تويتر الاحتياطي : reem78alfla,NaN,13141,187,2011-01-30,en,und,#اسقاط_القروض_لليوم_103,2019-03-17 21:59,Twitter for iPhone,NaN,NaN,1.107400e+18,False,NaN,NaN,absent,absent,0.0,0.0,1.0,3.0,['اسقاط_القروض_لليوم_103'],[],[],#اسقاط_القروض_لليوم_103
5,1174386089659965440,358533297,نواف.,Nawaf_UTD7,Khobar,@OnSideAr,NaN,6066,903,2011-08-20,en,ar,الحمدالله وأخيراً ضبط الحظ 😍 https://t.co/L9JSLtuFLL,2019-09-18 18:13,Twitter for iPhone,NaN,NaN,NaN,False,NaN,NaN,absent,absent,0.0,3.0,0.0,0.0,[],[],[],الحمدالله وأخيراً ضبط الحظ 😍
6,1107018391850807296,358533297,نواف.,Nawaf_UTD7,Khobar,@OnSideAr,NaN,6066,903,2011-08-20,en,ar,سويت كل شي ياماتيتش 💔,2019-03-16 20:38,Twitter for iPhone,NaN,NaN,NaN,False,NaN,NaN,absent,absent,0.0,1.0,0.0,0.0,[],[],[],سويت كل شي ياماتيتش 💔
7,1115299968733143040,358533297,نواف.,Nawaf_UTD7,Khobar,@OnSideAr,NaN,6066,903,2011-08-20,en,ar,جوفينكو وش قاعد يسوي!!,2019-04-08 17:06,Twitter for iPhone,NaN,NaN,NaN,False,NaN,NaN,absent,absent,0.0,0.0,0.0,0.0,[],[],[],جوفينكو وش قاعد يسوي!!
8,1143516030297395201,

## Rmove tags and links

In [130]:
import re

def remove_tags_and_links(text):
    # Remove mentions
    text = re.sub(r'@\w+', '', text)

    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    return text

tweets['cleaned_text'] = tweets['tweet_text'].apply(remove_tags_and_links)

## Embedding Example

In [123]:
english_sentences = set([
        "The quick brown fox jumps over the lazy dog.",
        "Tomorrow is another day.",
        "The sun rises in the east.",
        "Life is beautiful.",
        "Birds sing in the morning.",
        "The world is round.",
        "Time flies when you're having fun.",
        "All that glitters is not gold.",
        "Actions speak louder than words.",
        "Love conquers all.",
        "Knowledge is power.",
        "Beauty is in the eye of the beholder.",
        "Where there's smoke, there's fire.",
        "Rome wasn't built in a day.",
        "Money can't buy happiness.",
        "Laughter is the best medicine.",
        "Two heads are better than one.",
        "An apple a day keeps the doctor away.",
        "Hindsight is 20/20.",
        "The early bird catches the worm.",
        "Every cloud has a silver lining.",
        "A penny for your thoughts.",
        "A watched pot never boils.",
        "You can't judge a book by its cover.",
        "When in Rome, do as the Romans do.",
        "Fortune favors the bold.",
        "Where there is a will, there is a way.",
        "Practice makes perfect.",
        "Don't cry over spilled milk.",
        "The grass is always greener on the other side.",
        "There's no place like home.",
        "When the cat's away, the mice will play.",
        "Necessity is the mother of invention.",
        "The more, the merrier.",
        "Better late than never.",
        "A friend in need is a friend indeed.",
        "A rolling stone gathers no moss.",
        "It's a piece of cake.",
        "The pot calling the kettle black.",
        "Don't count your chickens before they hatch.",
        "A picture is worth a thousand words.",
        "When one door closes, another one opens.",
        "Beggars can't be choosers.",
        "A stitch in time saves nine.",
        "The squeaky wheel gets the grease.",
        "Home is where the heart is.",
        "People who live in glass houses shouldn't throw stones.",
        "Silence is golden.",
        "There's no smoke without fire.",
        "The proof is in the pudding.",
        "Absence makes the heart grow fonder.",
    ])
english_sentences = list(english_sentences)

In [124]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

embeddings = model.encode(english_sentences, convert_to_tensor=True)

#Compute cosine-similarities for each sentence with each other sentence
cosine_scores = util.cos_sim(embeddings, embeddings)

#Find the pairs with the highest cosine similarity scores
pairs = []
for i in range(len(cosine_scores)-1):
    for j in range(i+1, len(cosine_scores)):
        pairs.append({'index': [i, j], 'score': cosine_scores[i][j]})

#Sort scores in decreasing order
pairs = sorted(pairs, key=lambda x: x['score'], reverse=True)

for pair in pairs[0:5]:
    i, j = pair['index']
    print("{} \t\t {} \t\t Score: {:.4f}".format(english_sentences[i], english_sentences[j], pair['score']))
    print(i, j)

Where there's smoke, there's fire. 		 There's no smoke without fire. 		 Score: 0.8695
15 50
Home is where the heart is. 		 There's no place like home. 		 Score: 0.6953
10 27
Life is beautiful. 		 Beauty is in the eye of the beholder. 		 Score: 0.6411
6 12
Money can't buy happiness. 		 Beggars can't be choosers. 		 Score: 0.5800
29 44
The grass is always greener on the other side. 		 Every cloud has a silver lining. 		 Score: 0.5727
20 45


In [125]:
chinese_sentences = [
    "复杂的数学问题需要深入研究。",
    "他的论文包含了深刻的哲学思考。",
    "这个项目的成功需要协作和创新。",
    "他们在沙漠中度过了漫长而艰苦的旅程。",
    "这个机器学习算法需要大量的训练数据。",
    "这个方案的实施可能会遇到各种挑战。",
    "在高峰期，城市交通非常拥挤。",
    "这是一本充满难以理解的科学术语的书。",
    "他们的研究涉及到复杂的神经生物学。",
    "这个问题的答案并不明显。",
    "这个理论需要深入的逻辑推理。",
    "这个概念对于初学者来说可能有点抽象。",
    "这个算法的时间复杂度非常高。",
    "在这个实验中，精确度非常关键。",
    "这个课程的难度逐渐增加。",
    "他们的发现在科学界引起了广泛的争议。",
    "这个地区的气候条件非常恶劣。",
    "这个词汇表包括了许多生僻的词汇。",
    "在这个研究领域，仍然存在很多未解之谜。",
    "这个机器人的设计涉及到复杂的工程问题。",
    "这个方案的可行性尚未得到充分验证。",
    "这本书的内容非常深奥。",
    "这个论题涉及到众多的法律问题。",
    "这个数学难题花费了许多数学家多年时间来解决。",
    "他们的研究要求高度的专业知识。",
    "这个化学实验需要严格的控制条件。",
    "这个艺术品的价值令人难以估计。",
    "在这个领域，前沿研究非常具有挑战性。",
    "这个问题的解决方案仍然是一个谜。",
    "这个建筑项目的复杂性令人震惊。",
    "这个理论的数学证明非常复杂。",
    "这个电影的情节扑朔迷离。",
    "这个歌曲的歌词充满了隐喻。"
]



In [127]:
embeddings = model.encode(chinese_sentences, convert_to_tensor=True)

#Compute cosine-similarities for each sentence with each other sentence
cosine_scores = util.cos_sim(embeddings, embeddings)

#Find the pairs with the highest cosine similarity scores
pairs = []
for i in range(len(cosine_scores)-1):
    for j in range(i+1, len(cosine_scores)):
        pairs.append({'index': [i, j], 'score': cosine_scores[i][j]})

#Sort scores in decreasing order
pairs = sorted(pairs, key=lambda x: x['score'], reverse=True)

for pair in pairs[0:5]:
    i, j = pair['index']
    print("{} \t\t {} \t\t Score: {:.4f}".format(chinese_sentences[i], chinese_sentences[j], pair['score']))
    print(i, j)

复杂的数学问题需要深入研究。 		 这个理论的数学证明非常复杂。 		 Score: 0.8437
0 30
复杂的数学问题需要深入研究。 		 这个数学难题花费了许多数学家多年时间来解决。 		 Score: 0.8123
0 23
在这个研究领域，仍然存在很多未解之谜。 		 这个问题的解决方案仍然是一个谜。 		 Score: 0.7852
18 28
这个问题的答案并不明显。 		 这个问题的解决方案仍然是一个谜。 		 Score: 0.7788
9 28
这个理论需要深入的逻辑推理。 		 这个理论的数学证明非常复杂。 		 Score: 0.7678
10 30


In [99]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

sample_tweets = tweets[(tweets['tweet_language'] == 'en') & (tweets['is_retweet']==False)]['tweet_text'][:100]

embeddings = model.encode(list(sample_tweets), convert_to_tensor=True)

#Compute cosine-similarities for each sentence with each other sentence
cosine_scores = util.cos_sim(embeddings, embeddings)

#Find the pairs with the highest cosine similarity scores
pairs = []
top_pair = pd.DataFrame(columns=["Tweet 1", "Tweet 2", "Score"])
for i in range(len(cosine_scores)-1):
    for j in range(i+1, len(cosine_scores)):
      if cosine_scores[i][j] < 0.7:
        pairs.append({'index': [i, j], 'score': cosine_scores[i][j]})

#Sort scores in decreasing order
pairs = sorted(pairs, key=lambda x: x['score'], reverse=True)


for pair in pairs[0:20]:
    i, j = pair['index']
    top_pair = top_pair.append({"Tweet 1": list(sample_tweets)[i], "Tweet 2": list(sample_tweets)[j], "Score": pair['score']}, ignore_index=True)


/var/folders/5t/drh_ys0122sf_3q7npyxb3mh0000gn/T/ipykernel_45579/2272362689.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top_pair = top_pair.append({"Tweet 1": list(sample_tweets)[i], "Tweet 2": list(sample_tweets)[j], "Score": pair['score']}, ignore_index=True)
/var/folders/5t/drh_ys0122sf_3q7npyxb3mh0000gn/T/ipykernel_45579/2272362689.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top_pair = top_pair.append({"Tweet 1": list(sample_tweets)[i], "Tweet 2": list(sample_tweets)[j], "Score": pair['score']}, ignore_index=True)
/var/folders/5t/drh_ys0122sf_3q7npyxb3mh0000gn/T/ipykernel_45579/2272362689.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top_pair = top_pair.append({"Tweet 1": list(sample_tweets)[i],

In [100]:
top_pair

,Tweet 1,Tweet 2,Score
0,My instagram 🦋🦋 https://t.co/uq1GD9XDpp,Ander 💔💔 https://t.co/DgVJ5XwhDP,tensor(0.6999)
1,@RepJohnRose Houthis are terrorist and we should all stop them before it is too late,@RepJohnRose The Houthi militia has ballistic missiles which make the region and the US allies in a dangerous situation\n\n#WashTimesOpEd\n\n https://t.co/Au9UE69pHC,tensor(0.6998)
2,My instagram 🦋🦋 https://t.co/uq1GD9XDpp,Beauty 😍#sweety https://t.co/gAwy7aD7ta,tensor(0.6971)
3,@blueabaya @R_NS55 @AlArabiya beautiful thing,@Ziadx15 it’s beautiful,tensor(0.6966)
4,@RepJohnRose In Yemen there are terrorists supported by Iran and are very dangerous,@RepJohnRose The Houthi militia has ballistic missiles which make the region and the US allies in a dangerous situation\n\n#WashTimesOpEd\n\n https://t.co/Au9UE69pHC,tensor(0.6959)
5,Beauty 😍#sweety https://t.co/gAwy7aD7ta,@blueabaya @R_NS55 @AlArabiya beautiful thing,tensor(0.6905)
6,@RepJohnRose The world must be wary cause of the danger of the Huthi group.,@RepJohnRose The Houthi militia has ballistic missiles which make the region and the US allies in a dangerous situation\n\n#WashTimesOpEd\n\n https://t.co/Au9UE69pHC,tensor(0.6829)
7,Ander 💔💔 https://t.co/DgVJ5XwhDP,1:05pm 🎶 for you — I love your choice for me 🎼🖤 https://t.co/XpGUpG7fGO,tensor(0.6801)
8,Im back 😊🇰🇼❤️ #اسقاط_القروض_لليوم_111 https://t.co/ytTy0rz9yy,We are waiting 😍💙💙 https://t.co/eXJqYBpQRw,tensor(0.6771)
9,Beauty 😍#sweety https://t.co/gAwy7aD7ta,@tess_josephine Good boy https://t.co/vGDOIKyz04,tensor(0.6758)


In [103]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

sample_tweets = tweets[(tweets['tweet_language'] == 'en') & (tweets['is_retweet']==False)]['tweet_text'][:500]

embeddings = model.encode(list(sample_tweets), convert_to_tensor=True)

#Compute cosine-similarities for each sentence with each other sentence
cosine_scores = util.cos_sim(embeddings, embeddings)

#Find the pairs with the highest cosine similarity scores
pairs = []
top_pair = pd.DataFrame(columns=["Tweet 1", "Tweet 2", "Score"])
for i in range(len(cosine_scores)-1):
    for j in range(i+1, len(cosine_scores)):
      if cosine_scores[i][j] < 0.7:
        pairs.append({'index': [i, j], 'score': cosine_scores[i][j]})

#Sort scores in decreasing order
pairs = sorted(pairs, key=lambda x: x['score'], reverse=True)


for pair in pairs[0:20]:
    i, j = pair['index']
    top_pair = top_pair.append({"Tweet 1": list(sample_tweets)[i], "Tweet 2": list(sample_tweets)[j], "Score": pair['score']}, ignore_index=True)


/var/folders/5t/drh_ys0122sf_3q7npyxb3mh0000gn/T/ipykernel_45579/1730587869.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top_pair = top_pair.append({"Tweet 1": list(sample_tweets)[i], "Tweet 2": list(sample_tweets)[j], "Score": pair['score']}, ignore_index=True)
/var/folders/5t/drh_ys0122sf_3q7npyxb3mh0000gn/T/ipykernel_45579/1730587869.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top_pair = top_pair.append({"Tweet 1": list(sample_tweets)[i], "Tweet 2": list(sample_tweets)[j], "Score": pair['score']}, ignore_index=True)
/var/folders/5t/drh_ys0122sf_3q7npyxb3mh0000gn/T/ipykernel_45579/1730587869.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top_pair = top_pair.append({"Tweet 1": list(sample_tweets)[i],

In [102]:
top_pair

,Tweet 1,Tweet 2,Score
0,@SenShelby @POTUS @FHFA @BankingGOP Houthis are very dangerous and could be the next AlQaeda if they attacked USA,@RepJohnRose Houthis are terrorist and we should all stop them before it is too late,tensor(0.6875)
1,My instagram 🦋🦋 https://t.co/uq1GD9XDpp,Beauty 😍#sweety https://t.co/gAwy7aD7ta,tensor(0.6857)
2,@SenShelby @POTUS @FHFA @BankingGOP Houthis are very dangerous and could be the next AlQaeda if they attacked USA,@RepJohnRose The Houthi militia has ballistic missiles which make the region and the US allies in a dangerous situation\n\n#WashTimesOpEd\n\n https://t.co/Au9UE69pHC,tensor(0.6830)
3,"@RepJohnRose Thank you Peter for this effort, it is necessary to warn the world about a dangerous threat like Houthi.\n\nhttps://t.co/JEtV39D6oE",@RepJohnRose The Houthi militia has ballistic missiles which make the region and the US allies in a dangerous situation\n\n#WashTimesOpEd\n\n https://t.co/Au9UE69pHC,tensor(0.6766)
4,@blueabaya @R_NS55 @AlArabiya beautiful thing,@Ziadx15 it’s beautiful,tensor(0.6697)
5,@RepJohnRose @Varneyco @FoxBusiness I believe that the Houthis and Hezbollah is more dangerous than Al Qaida and ISIS because the took More than 20 years to prepare their activities\n\nhttps://t.co/sIpW2RE1wL,@RepJohnRose The Houthi militia has ballistic missiles which make the region and the US allies in a dangerous situation\n\n#WashTimesOpEd\n\n https://t.co/Au9UE69pHC,tensor(0.6690)
6,@RepJohnRose Houthis are terrorist and we should all stop them before it is too late,"@RepJohnRose Thank you Peter for this effort, it is necessary to warn the world about a dangerous threat like Houthi.\n\nhttps://t.co/JEtV39D6oE",tensor(0.6652)
7,@RepJohnRose @Varneyco @FoxBusiness I believe that the Houthis and Hezbollah is more dangerous than Al Qaida and ISIS because the took More than 20 years to prepare their activities\n\nhttps://t.co/sIpW2RE1wL,@RepJohnRose In Yemen there are terrorists supported by Iran and are very dangerous,tensor(0.6558)
8,@EpicEnix roll it again,@EpicEnix Unlimited power,tensor(0.6534)
9,@SenShelby @POTUS @FHFA @BankingGOP Houthis are very dangerous and could be the next AlQaeda if they attacked USA,@RepJohnRose In Yemen there are terrorists supported by Iran and are very dangerous,tensor(0.6523)


In [132]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
start_time = time.time()
sample_tweets = tweets[0:10000]['cleaned_text']
embeddings = model.encode(list(sample_tweets), convert_to_tensor=True)
end_time = time.time()
print("MiniLM Embedding Time: {}".format(i, end_time-start_time))

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
start_time = time.time()
sample_tweets = tweets[0:10000]['cleaned_text']
embeddings = model.encode(list(sample_tweets), convert_to_tensor=True)
end_time = time.time()
print("mpnet Embedding Time: {}".format(i, end_time-start_time))
